# Assignment 2. Programming Intelligent Agents
MTY - A01152534 - Jorge Antonio Ayala Urbina  
MTY - Datos Ale  
MTY - A01037093 - Miguel Angel Cruz Gomez 

In [1]:
from agents import *
import random

# Create things

# Treasure1 thing
class T(Thing):
    pass

# Treasure2 thing
class t(Thing):
    pass

#Reusable tool thing
class H(Thing):
    pass

#Disposable tool thing
class h(Thing):
    pass

#Wall thing
class w(Thing):
    pass

#In this enviroment, the agent can see it all
class IslandDay(Environment):
    
    #Flag to be activated when there are no more treasures or no treasures that can be gathered
    #agent_no_goals = False
    
    #As the environment if fully observable, the percept should contain everything found in the environment
    def percept(self, agent):
        in_existence = self.things
        in_posession = agent.holding
        at_position = self.list_things_at(agent.location)
        perception = [in_existence, in_posession, at_position]
        return perception
    
    
    
    def execute_action(self, agent, action):

        if action == "Move":
            location = agent.location
            possMoves = [location]
            
            moveUp = (location[0],location[1]+1)
            moveDown = (location[0],location[1]-1)
            moveRight = (location[0]+1,location[1])
            moveLeft = (location[0]-1,location[1])
                
            if (moveUp[1] < 6 and len(self.list_things_at(moveUp, tclass=w)) == 0):
                possMoves.append(moveUp)
            if (moveDown[1] >= 0 and len(self.list_things_at(moveDown, tclass=w)) == 0):
                possMoves.append(moveDown)
            if (moveRight[0] < 6 and len(self.list_things_at(moveRight, tclass=w)) == 0):
                possMoves.append(moveRight)
            if (moveLeft[0] >= 0 and len(self.list_things_at(moveLeft, tclass=w)) == 0):
                possMoves.append(moveLeft)
                
            direction = possMoves.index(random.choice(possMoves))
            agent.move(possMoves[direction])  
            
        # If the action is to pick up reusable tool, run Gresure and remove the thing from the environment
        # Appending the object to the agent is done at the agent class. Same for all the following methods
        
        elif action == "Greuse":
            items = self.list_things_at(agent.location, tclass = H)
            holding = agent.get_held_things(tclass = H)
            if len(items) != 0:
                agent.Greuse(items[0])
                self.delete_thing(items[0])
                
        elif action == "Gdispos":
            items = self.list_things_at(agent.location, tclass = h)
            holding = agent.get_held_things(tclass = h)
            if len(items) != 0:
                agent.Gdispos(items[0])
                self.delete_thing(items[0])
                
        elif action == "GTreas1":
            items = self.list_things_at(agent.location, tclass = T)
            holding = agent.get_held_things(tclass = H)
            if (len(items) != 0 and len(holding) != 0):
                agent.GTreas1(items[0])
                self.delete_thing(items[0])
        elif action == "GTreas2":
            items = self.list_things_at(agent.location, tclass = t)
            holding = agent.get_held_things(tclass = h)
            if (len(items) != 0 and len(holding) != 0):
                agent.GTreas2(items[0])
                self.delete_thing(items[0])
        else:
            location = agent.location
            possMoves = [location]
            
            moveUp = (location[0],location[1]+1)
            moveDown = (location[0],location[1]-1)
            moveRight = (location[0]+1,location[1])
            moveLeft = (location[0]-1,location[1])
                
            if (moveUp[1] < 6 and len(self.list_things_at(moveUp, tclass=w)) == 0):
                possMoves.append(moveUp)
            if (moveDown[1] >= 0 and len(self.list_things_at(moveDown, tclass=w)) == 0):
                possMoves.append(moveDown)
            if (moveRight[0] < 6 and len(self.list_things_at(moveRight, tclass=w)) == 0):
                possMoves.append(moveRight)
            if (moveLeft[0] >= 0 and len(self.list_things_at(moveLeft, tclass=w)) == 0):
                possMoves.append(moveLeft)
                
            direction = possMoves.index(random.choice(possMoves))
            agent.move(possMoves[direction])  
            
            
                  
    def is_done(self):
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents 
    #agent_no_goals

    def step(self):
        if not self.is_done():
#             print(self.agents[0].holding)
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        "Run the Environment for given number of time steps."
        for step in range(steps):
            if self.is_done():
                return
            self.step()
                  
                  
       

rules = ['move','Greuse', 'Gdispos', 'GTreas1', 'GTreas2', 'NoOp']
class Rule():
    

In [2]:

class Hunter(Agent):
    
    Agent.performance = 50
    
    def move(self, direc):
        self.performance -= 1
        prevLoc = self.location
        self.location = (direc[0],direc[1])
        print('Hunter: Moved from {} to {}'.format(prevLoc,self.location))
        
    def get_held_things(self, tclass=Thing):
        #Returns the thing held by the agent
        return [thing for thing in self.holding
                if isinstance(thing, tclass)]
    
    #If Greuse is called, the thing is appended to self.holding
    def Greuse(self, thing):
        if isinstance(thing, H):
            print("Hunter: Grabbed reusable tool at {}.".format(self.location))
            self.holding.append(thing)
            return True
        return False
    
    def Gdispos(self, thing):
#         print(isinstance(thing,h))
        if isinstance(thing, h):
            print("Hunter: Grabbed disposable tool at {}.".format(self.location))
            self.holding.append(thing)
            return True
        return False
    
    def GTreas1(self, thing):
        
        if isinstance(thing, T):
            print("Hunter: Grabbed a Treasure1 at {}.".format(self.location))
            self.holding.append(thing)
            self.performance +=20
            print("Hey! my performance is: {}".format(self.performance))
            return True
        return False
    
    #Removes disposable tool after use
    def GTreas2(self, thing):
        if isinstance(thing, t):
            print("Hunter: Grabbed a Treasure2 at {}.".format(self.location))
            self.holding.append(thing)
            self.performance += 40
            print("Hey! my performance is: {}".format(self.performance))
            for tool in self.holding:
                if isinstance(tool, h):
                    self.holding.remove(tool)
                    print("Disposable tool has been lost")
                    break
            return True
        return False

In [3]:
def interpret_input(percept):
    
    in_environmentAux = percept[0]
    in_environment = in_environmentAux.copy()
    in_agent = percept[1]
    in_location = percept[2]
    agent = in_location[0]
    #Restrict what the agent can see.
    for thing in in_environment:
        if (abs(thing.location[0] - agent.location[0])> 1) or ((abs(thing.location[1] - agent.location[1])> 1)):
            in_environment.remove(thing)
    
    agent_has_H = False
    agent_has_h = False
    
    if len(in_agent) != 0:
        for thing in in_agent:
            if isinstance(thing,h):
                agent_has_h = True
            elif isinstance(thing,H):
                agent_has_H = True
            
    
    collectables_exist = False
    
    for thing in in_environment:
        if (isinstance(thing, T) or isinstance(thing, t) or isinstance(thing, H) or isinstance(thing, t)):
            collectables_exist = True
            break
    
    #Check existences
    T_exists = False
    t_exists = False
    H_exists = False
    h_exists = False
    
    for thing in in_environment:
        if isinstance(thing, T):
            T_exists = True
            break
            
    for thing in in_environment:
        if isinstance(thing, t):
            t_exists = True
            break
            
    for thing in in_environment:
        if isinstance(thing, H):
            H_exists = True
            break
            
    for thing in in_environment:
        if isinstance(thing, h):
            h_exists = True
            break
            
    for thing in in_agent:
        if isinstance(thing, H):
            H_exists = True
            break
            
    for thing in in_agent:
        if isinstance(thing, h):
            h_exists = True
            break
    
    if len(in_location) != 1:
        
        if isinstance(in_location[1], H):
            print("Hey! I found a Greuse")
            return 'Greuse'
        elif isinstance(in_location[1], h):
            print("Hey! I found a Gdispos at: {}".format(in_location[0].location))
            return 'Gdispos'
        elif (isinstance(in_location[1], T) and agent_has_H):
            print("Hey! I found a Treasure")
            return 'GTreas1'
        elif (isinstance(in_location[1], t) and agent_has_h):
            print("Hey! I found a treasure")
            return 'GTreas2'
        elif (isinstance(in_location[1], T) and not(agent_has_H)):
            print("Can't pick up T since I have no H, so I'm moving ")
            return 'Move'
        elif (isinstance(in_location[1], t) and not (agent_has_h)):
            print("Can't pick up t since I have no h, so I'm moving ")
            return 'Move'
        else:
            print("Did'nt move")
            return 'NoOp'
    if len(in_location) == 1:
        if((T_exists and H_exists) or (t_exists and h_exists)):
            return 'Move'
        else:
#             print("Did'nt move")
            return 'NoOp'

        
class Rules():
    def __init__(self, action = ''):
        self.action = action
        
    def matches(self, a_state):
            return self.action == a_state
        
        
Explore = Rules("Move")
ReusableTool = Rules("Greuse")
DisposableTool = Rules("Gdispos")
Treasure1 = Rules("GTreas1")
Treasure2 = Rules("GTreas2")
NothingToDo = Rules("NoOp")

rules = [Explore, ReusableTool, DisposableTool, Treasure1, Treasure2, NothingToDo]

# print("Explore:")
# print(Explore)
# print("Greuse:")


In [4]:
island1 = IslandDay()
hunter_bob = Hunter(SimpleReflexAgentProgram(rules, interpret_input))

H1 = H()
T1 = T()
T2 = T()
h1 = h()
h2 = h()
t1 = t()
t2 = t()


island1.add_thing(hunter_bob, (0,0))
island1.add_thing(H1, (1,1))
island1.add_thing(T1, (1,3))
island1.add_thing(T2, (1,4))
island1.add_thing(h1, (1,2))
island1.add_thing(h2, (0,2))
island1.add_thing(t1, (4,1))
island1.add_thing(t2, (4,2))

island1.things


[<Hunter>, <H>, <T>, <T>, <h>, <h>, <t>, <t>]

In [17]:
island1.run(30)

Hunter: Moved from (3, 3) to (3, 2)
Hunter: Moved from (3, 2) to (3, 3)
Hunter: Moved from (3, 3) to (2, 3)
Hunter: Moved from (2, 3) to (2, 3)
Hunter: Moved from (2, 3) to (3, 3)
Hunter: Moved from (3, 3) to (2, 3)
Hunter: Moved from (2, 3) to (2, 2)
Hunter: Moved from (2, 2) to (2, 2)
Hunter: Moved from (2, 2) to (1, 2)
Hunter: Moved from (1, 2) to (1, 2)
Hunter: Moved from (1, 2) to (2, 2)
Hunter: Moved from (2, 2) to (1, 2)
Hunter: Moved from (1, 2) to (1, 3)
Hey! I found a Treasure
Hunter: Grabbed a Treasure1 at (1, 3).
Hey! my performance is: -49
Hunter: Moved from (1, 3) to (0, 3)
Hunter: Moved from (0, 3) to (0, 2)
Hey! I found a Gdispos at: (0, 2)
Hunter: Grabbed disposable tool at (0, 2).
Hunter: Moved from (0, 2) to (0, 3)
Hunter: Moved from (0, 3) to (0, 3)
Hunter: Moved from (0, 3) to (1, 3)
Hunter: Moved from (1, 3) to (0, 3)
Hunter: Moved from (0, 3) to (0, 2)
Hunter: Moved from (0, 2) to (0, 1)
Hunter: Moved from (0, 1) to (0, 2)
Hunter: Moved from (0, 2) to (1, 2)
Hunt

In [18]:
hunter_bob.holding

[<H>, <t>, <T>, <T>, <h>]

In [19]:
island1.things

[<Hunter>, <t>]